# Mosaic Integration (HT)

config

In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
import numpy as np
import SpaMode
import copy
import scipy
import anndata as ad
import argparse

from typing import Optional
from scanpy.pp import combat
from SpaMode.preprocess import clr_normalize_each_cell, pca, lsi
from SpaMode.preprocess import construct_neighbor_graph
from SpaMode.utils import clustering, peak_sets_alignment, gene_sets_alignment
from cal_matrics import eval
from SME.Svae import Train_Smoe
from scipy.sparse import coo_matrix
from sklearn.neighbors import kneighbors_graph
from scipy.sparse import csr_matrix

import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import laplacian
from scipy.spatial import procrustes
from sklearn.metrics.pairwise import cosine_similarity

import scipy.sparse as sps
from sklearn.metrics import adjusted_rand_score, roc_auc_score, f1_score

Data Process

In [ ]:
# Environment configuration. SpatialGlue pacakge can be implemented with either CPU or GPU. GPU acceleration is highly recommend for imporoved efficiency.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def binarize(Xs, bin_thr=0):
    rs = []
    for X in Xs:
        X = copy.deepcopy(X.A) if sps.issparse(X) else copy.deepcopy(X)
        X = np.where(X>bin_thr, 1, 0)
        rs.append(X)
    return rs

def eval_AUC_all(gt_X, pr_X, bin_thr=1):
    gt_X = binarize([gt_X], bin_thr)[0].flatten()
    pr_X = pr_X.flatten()
    auroc = roc_auc_score(gt_X, pr_X)
    return auroc

def PCCs(gt_X, pr_X):
    def _compute_pcc(x, y, axis):
        x_centered = x - x.mean(axis=axis, keepdims=True)
        y_centered = y - y.mean(axis=axis, keepdims=True)
        cov = (x_centered * y_centered).sum(axis=axis)
        std_x = np.sqrt((x_centered  **  2).sum(axis=axis))
        std_y = np.sqrt((y_centered  **  2).sum(axis=axis))
        denom = std_x * std_y
        denom[denom == 0] = np.nan
        return cov / denom

    if sps.issparse(gt_X):
        gt_X = gt_X.A  
    if sps.issparse(pr_X):
        pr_X = pr_X.A

    pcc_cell = _compute_pcc(gt_X, pr_X, axis=1)

    return pcc_cell, 0


def CMD(pr_X, gt_X):
    zero_mask = (~pr_X.any(axis=1)) | (~gt_X.any(axis=1))
    rm_p = zero_mask.mean()
    if rm_p >= 0.05:
        print(f'Warning: too many rows ({rm_p * 100:.1f}%) with all zeros')

    pr_array = pr_X[~zero_mask].astype(np.float32)
    gt_array = gt_X[~zero_mask].astype(np.float32)

    def fast_corr(X):
        X_centered = X - X.mean(axis=1, keepdims=True)
        cov = X_centered @ X_centered.T
        std = np.sqrt(np.sum(X_centered ** 2, axis = 1, keepdims = True))
        std[std == 0] = 1  
        return cov / (std @ std.T)

    corr_pr = fast_corr(pr_array)
    corr_gt = fast_corr(gt_array)

    x = np.sum(corr_pr * corr_gt.T)  
    y = np.sqrt(np.sum(corr_pr ** 2)) *np.sqrt(np.sum(corr_gt ** 2))

    cmd = 1 - x / (y + 1e-8)
    return cmd

def reconstruct_missing_modality(existing_m1_emb, ref_m1_emb, existing_m2, A1, A2, recon_s2_m2=None, imput_pca=False):

    existing_m1_emb = aggregate_neighborhood_features(existing_m1_emb, A1)
    ref_m1_emb = aggregate_neighborhood_features(ref_m1_emb, A2)

    if imput_pca:

        nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='euclidean')
        nn.fit(existing_m1_emb)

        _, indices = nn.kneighbors(ref_m1_emb)
        indices = indices.ravel()

        imput_m2 = existing_m2[indices]

        imput_m1 = existing_m1_emb[indices]

        erro = torch.tensor(1 - cosine_similarity(imput_m1, ref_m1_emb)).cuda()
        erro = erro.unsqueeze(1)
        imput_m2 = torch.tensor(imput_m2).cuda()
        recon_s2_m2 = torch.tensor(recon_s2_m2).cuda()
        imput = F.normalize(((1 - erro) * imput_m2 + erro * recon_s2_m2), p=2, eps=1e-12, dim=1)

    else:
        K = 25  

        nn = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='euclidean')
        nn.fit(existing_m1_emb)

        _, indices = nn.kneighbors(ref_m1_emb) 

        if sps.issparse(existing_m2):
            indices_flat = indices.ravel()  
            imput_m2 = existing_m2[indices_flat, :].toarray()  
            imput_m2 = imput_m2.reshape(-1, K, imput_m2.shape[1])  
        else:
            imput_m2 = existing_m2[indices]

        imput_m2 = np.mean(imput_m2, axis=1)  

        imput = imput_m2

    return imput, existing_m1_emb, ref_m1_emb


def cosine_similarity(a, b):
    # a, b: shape [..., D]
    a_norm = a / np.linalg.norm(a, axis=-1, keepdims=True)
    b_norm = b / np.linalg.norm(b, axis=-1, keepdims=True)
    return np.sum(a_norm * b_norm, axis=-1)  


def aggregate_neighborhood_features(R, A, aggregation='mean'):

    N = R.shape[0]
    R_agg = []
    A_sparse = csr_matrix(A.detach().cpu().numpy())  

    for i in range(N):
        neighbors = A_sparse[i].indices
        if aggregation == 'mean':
            feat = R[neighbors].mean(axis=0)
        elif aggregation == 'sum':
            feat = R[neighbors].sum(axis=0)
        elif aggregation == 'concat':
            feat = R[neighbors].flatten()  
            feat = feat[:30]  
        R_agg.append(feat)

    return np.array(R_agg)


def data_preprocessing(adata_omics1, adata_omics2):

    adata_omics1.var_names_make_unique()
    adata_omics2.var_names_make_unique()

    # RNA
    sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
    sc.pp.normalize_total(adata_omics1, target_sum=1e4)
    sc.pp.log1p(adata_omics1)
    sc.pp.scale(adata_omics1, max_value=100)

    adata_omics1_high = adata_omics1[:, adata_omics1.var['highly_variable']]
    adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=30)

    # Protein
    adata_omics2 = clr_normalize_each_cell(adata_omics2)
    sc.pp.scale(adata_omics2)
    adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=30)

    print(adata_omics1)
    print(adata_omics2)

    return adata_omics1, adata_omics2






Mosaic Integration

In [ ]:


# Fix random seed
from SME.preprocess import fix_seed
random_seed = 2025
fix_seed(random_seed)

file_paths = {
    '1': ['/root/autodl-tmp/Data/Human_tonsil/slice1/s1_adata_rna.h5ad',
              '/root/autodl-tmp/Data/Human_tonsil/slice1/s1_adata_adt.h5ad'],
    '2': ['/root/autodl-tmp/Data/Human_tonsil/slice2/s2_adata_rna.h5ad',
              '/root/autodl-tmp/Data/Human_tonsil/slice2/s2_adata_adt.h5ad'],
    '3': ['/root/autodl-tmp/Data/Human_tonsil/slice3/s3_adata_rna.h5ad',
              '/root/autodl-tmp/Data/Human_tonsil/slice3/s3_adata_adt.h5ad'],
}

GT_paths = {
    '1': '/root/autodl-tmp/0222/Results/HT/slice-1/HT-slice-1-GT.txt',
    '2': '/root/autodl-tmp/0222/Results/HT/slice-2/HT-slice-2-GT.txt',
    '3': '/root/autodl-tmp/0222/Results/HT/slice-3/HT-slice-3-GT.txt',
}

adata_omics1, adata_omics2 = [sc.read_h5ad(fp) for fp in file_paths['1']]

adata_omics1_cp = copy.deepcopy(adata_omics1)
adata_omics2_cp = copy.deepcopy(adata_omics2)

adata_omics1, adata_omics2 = data_preprocessing(adata_omics1, adata_omics2)
adata = construct_neighbor_graph(adata_omics1, adata_omics2, datatype='10x')

import json
# 加载配置文件
with open("/root/autodl-tmp/0222/config/cfg_training.json", "r") as file:
    config = json.load(file)

from SME.SpaMode_Mosaic import Train_SpaMode
model = Train_SpaMode(adata, datatype='10x', device=device, Arg=config["HT"]['1'])
output = model.train()

adata_omics1.obsm['Smoe'] = output['Smoe']

# Imputation S2_ADT
adata_omics1_E15, adata_omics2_E15 = [sc.read_h5ad(fp) for fp in file_paths['2']]

adata_omics2_E15_cp = copy.deepcopy(adata_omics2_E15)

adata_omics1_E15, adata_omics2_E15 = data_preprocessing(adata_omics1_E15, adata_omics2_E15)
adata_E15 = construct_neighbor_graph(adata_omics1_E15, adata_omics1_E15, datatype='10x')
infer_results = model.imputation(adata_E15, 1)

imput_adt_pca, adata_omics1_spatial_smooth, adata_omics1_E15_spatial_smooth = reconstruct_missing_modality(
        adata_omics1.obsm['feat'],
        adata_omics1_E15.obsm['feat'],
        adata_omics2.obsm['feat'],
        model.adj_spatial_omics1,
        model.tmp_adj_spatial_omics1,
        F.normalize(
            infer_results['emb_recon_omics2'],
            p=2, eps=1e-12, dim=1),
        imput_pca=True
    )

adata_omics2_E15.obsm['feat'] = imput_adt_pca.detach().cpu().numpy()

adata_E15_imput = construct_neighbor_graph(adata_omics1_E15, adata_omics2_E15, datatype='10x')
model_mosaic = Train_SpaMode(adata_E15_imput, datatype='10x', device=device, Arg=config["HT"]['2'])
output = model_mosaic.train()
adata_omics1_E15.obsm['Smoe'] = output['Smoe']
# adata_omics1_E15.write_h5ad('/root/autodl-tmp/0222/Results/HT/Mosaic/Our/Slice_2.h5ad')






AnnData object with n_obs × n_vars = 4326 × 18085
    obs: 'lab', 'lab_lynn', 'src', 'final_annot'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'hvg', 'log1p'
    obsm: 'spatial', 'feat'
AnnData object with n_obs × n_vars = 4326 × 31
    obs: 'lab', 'lab_lynn', 'src', 'final_annot'
    var: 'gene_ids', 'feature_types', 'genome', 'mean', 'std'
    obsm: 'spatial', 'feat'
Spatial_neighbors = 3
Feature_neighbors = 20


/root/autodl-tmp/SpaMode_0903/SME/Svae_0318.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weight = F.softmax(differ_stack)


tensor([0.8057, 0.5797], device='cuda:0') tensor([0.5563, 0.4437], device='cuda:0')
optimizer:  SGD


 51%|█████     | 509/1000 [01:55<01:51,  4.41it/s, Biloss=0.191, adv_loss=0.271, kl_loss=0.897, moe_loss=0.0202, recon=18.6]  


Early Stop
Model training finished!

AnnData object with n_obs × n_vars = 4519 × 18085
    obs: 'lab', 'src', 'final_annot'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'hvg', 'log1p'
    obsm: 'spatial', 'feat'
AnnData object with n_obs × n_vars = 4519 × 31
    obs: 'lab', 'src', 'final_annot'
    var: 'gene_ids', 'feature_types', 'genome', 'mean', 'std'
    obsm: 'spatial', 'feat'
Spatial_neighbors = 3
Feature_neighbors = 20


...

Spatial_neighbors = 3
Feature_neighbors = 20


/root/autodl-tmp/SpaMode_0903/SME/Svae_0318.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weight = F.softmax(differ_stack)


tensor([0.7120, 0.8796], device='cuda:0') tensor([0.4582, 0.5418], device='cuda:0')
optimizer:  SGD


 55%|█████▍    | 549/1000 [02:21<01:56,  3.87it/s, Biloss=0.166, adv_loss=0.245, kl_loss=0.864, moe_loss=0.00209, recon=12.5] 

Early Stop
Model training finished!



Imputation

In [5]:

mix_omics1 = np.concatenate([adata_omics1.obsm['feat'], adata_omics2.obsm['feat']], axis=1)
mix_omics2 = np.concatenate([adata_omics1_E15.obsm['feat'], adata_omics2_E15.obsm['feat']], axis=1)

imput_adt_raw, _, _ = reconstruct_missing_modality(
        mix_omics1,
        mix_omics2,
        adata_omics2_cp.X,
        model.adj_spatial_omics1,
        model.tmp_adj_spatial_omics1,
)


adata_omics2_E15_imputed = copy.deepcopy(adata_omics2_E15_cp)
adata_omics2_E15_imputed.X = csr_matrix(imput_adt_raw)
adata_omics2_E15_imputed.write_h5ad('/root/autodl-tmp/0222/Results/HT/Imput/HT_S2_ADT_imput_our.h5ad')

print("-----------------Eval Imput ADT------------------")

gt_X = adata_omics2_E15_cp.X.toarray()
pr_X = imput_adt_raw

pcc_cell, pcc_adt = PCCs(gt_X, pr_X)

cmd_cell = CMD(pr_X, gt_X)
cmd_adt = CMD(pr_X.T, gt_X.T)

print('PCC cell:', np.mean(pcc_cell))
print('PCC RNA:', np.mean(pcc_adt))
print('Command cell:', cmd_cell)
print('Command RNA:', cmd_adt)

print("-----------------Eval Imput ADT------------------")

-----------------Eval Imput ADT------------------
PCC cell: 0.93754214
PCC RNA: 0.0
Command cell: 0.010368214880204674
Command RNA: 0.22684393721285268
-----------------Eval Imput ADT------------------
